In [13]:
# Using test data to build/prototype-out a ML model
    # Further restructuring data for ML input layers
        # Variable isolation and selection refinement
    # Reshaping, encoding, normalization
    # Input, layer, and output specification
    # Force keras/tensorflow to use GPU backend

In [14]:
# Imports
import pandas as pd
import numpy as np
import json
import datetime 
import matplotlib.pyplot as plt
%matplotlib inline

# Pymongo import for connection to local client DB
import pymongo
from pymongo import MongoClient

# ML Imports 
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM, GRU
from keras.models import load_model
from keras import backend as K

# Preprocessing Imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from keras.utils import to_categorical 

# Import to check check for GPU availability for tensorflow backend
from tensorflow.python.client import device_lib

In [15]:
# Mongodb connection for live feed of data into model
    # Implementation after verification of test data -> model prototype input layer working

In [16]:
# Verify GPU availability for tensorflow backend
print(device_lib.list_local_devices())
print("==============================================")
print(K.tensorflow_backend._get_available_gpus())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16169846431139365899
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9222031934
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13036165719127486442
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
['/job:localhost/replica:0/task:0/device:GPU:0']


In [17]:
# Read in test data for L2 orderbook state (bids + asks)
    # Read in test data for subsquent L2 orderbook update states (L2 updates to bid + asks)
snapshot_asks_df = pd.read_csv("test_data/snapshot_asks.csv")
snapshot_bids_df = pd.read_csv("test_data/snapshot_bids.csv")
l2update_df = pd.read_csv("test_data/l2update.csv", dtype ={'changes':object})
request_log_df= pd.read_csv("test_data/request_log.csv")

In [18]:
print(snapshot_asks_df.head())
print(snapshot_bids_df.head())

   side    price      size
0  sell  8014.00  0.006469
1  sell  8014.34  0.001000
2  sell  8015.00  0.021821
3  sell  8015.15  0.006000
4  sell  8015.19  0.023841
  side    price       size
0  buy  8013.99  12.482824
1  buy  8011.00  23.900000
2  buy  8010.99   3.501000
3  buy  8010.00   1.000000
4  buy  8008.00   0.001000


In [19]:
l2update_df.head()

,_id,time,side,price,size
0,5ad526dce3ae7141c0170d1a,2018-04-16T22:42:38.806Z,buy,7887.88,0.0015
1,5ad526dce3ae7141c0170d1b,2018-04-16T22:42:38.807Z,buy,7887.88,0.0015
2,5ad526dce3ae7141c0170d1c,2018-04-16T22:42:38.809Z,buy,7887.88,0.0015
3,5ad526dce3ae7141c0170d1e,2018-04-16T22:42:38.817Z,buy,7887.88,0.0015
4,5ad526dce3ae7141c0170d1f,2018-04-16T22:42:38.823Z,buy,7887.88,0.0015


In [20]:
request_log_df.head()

,Unnamed: 0,iso,epoch
0,request end,2018-04-16T22:44:37.747Z,1.523919e+09
1,request start,2018-04-16T22:42:37.724Z,1.523919e+09


In [ ]:
# Bid= buy
# Ask = sell

#L2 snapshot structure
    # [price,size]
    # 'side' added as part of structure for classification
#l2 updates structure
    # [side, price, size]

# Note on GDAX API about l2update structure:
    # size of "0" indicates the price level can be removed